In [ ]:
# default_exp covariables

# covariables

> Construcción de covariables para analizar delitos

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import os
import glob
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import requests

In [ ]:
# export
DATA_PATH = "datos/"
DOWNLOADS_PATH = "datos/descargas/"

## descarga_datos_covariables

In [ ]:
# export
def descarga_datos_covariables():
    """Descarga los archivos necesarios qe son demasiado grandes para el repositorio.
    
        - geometrías de manzanas
    """
    covariables_url = "https://www.dropbox.com/s/s49lb476wpwu2p1/covariables.gpkg?dl=1"
    r = requests.get(covariables_url, allow_redirects=True)
    open(DOWNLOADS_PATH + 'covariables.gpkg', 'wb').write(r.content)

In [ ]:
descarga_datos_covariables()

## get_diccionario_censo

In [ ]:
# export
def get_diccionario_censo():
    """Regresa un DataFrame con el diccionario de variables del censo."""
    dicionario = pd.read_csv("datos/diccionario_datos_ageb_urbana_09_cpv2020.csv", skiprows=3)
    diccionario = (dicionario
                   .drop(range(0,8))
                   .drop(columns='Núm.')
                   .reset_index(drop=True)
                   .rename({'Mnemónico':'Nombre del Campo'}, axis=1))
    return diccionario

In [ ]:
diccionario = get_diccionario_censo()
diccionario.head()

,Indicador,Descripción,Nombre del Campo,Rangos,Longitud
0,Población total,Total de personas que residen habitualmente en...,POBTOT,0...999999999,9
1,Población femenina,Total de mujeres que residen habitualmente en ...,POBFEM,0...999999999,9
2,Población masculina,Total de hombres que residen habitualmente en ...,POBMAS,0...999999999,9
3,Población de 0 a 2 años,Personas de 0 a 2 años de edad.,P_0A2,0…999999999,9
4,Población femenina de 0 a 2 años,Mujeres de 0 a 2 años de edad.,P_0A2_F,"0.,.999999999",9


## get_variables_censo

In [ ]:
# export
def get_variables_censo():
    """Regresa un DataFrame con las variables del censo a nivel manzana."""
    df = pd.read_csv("datos/censo_manzanas.zip", dtype={'CVEGEO':str, 'colonia_cve': 'Int64'})
    return df

In [ ]:
censo = get_variables_censo()
censo.head()

,CVEGEO,AMBITO,TIPOMZA,POBTOT,POBFEM,POBMAS,P_0A2,P_0A2_F,P_0A2_M,P_3YMAS,...,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,colonia_cve,cuadrante_id
0,0901000010898031,Urbana,Típica,93.0,56.0,37.0,4.0,NaN,3.0,89.0,...,15.0,16.0,6.0,3.0,0.0,0.0,7.0,0.0,1119,17.0
1,0901000012269024,Urbana,Típica,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1082,14.0
2,0901000011472068,Urbana,Típica,124.0,66.0,58.0,3.0,3.0,0.0,121.0,...,25.0,22.0,9.0,8.0,0.0,NaN,7.0,0.0,1030,11.0
3,0901000011824024,Urbana,Típica,340.0,177.0,163.0,12.0,8.0,4.0,327.0,...,69.0,56.0,29.0,14.0,NaN,NaN,25.0,NaN,1135,110.0
4,0901000012377004,Urbana,Típica,82.0,41.0,41.0,NaN,0.0,NaN,80.0,...,13.0,13.0,6.0,3.0,0.0,0.0,9.0,0.0,1081,18.0


## imputa_faltantes_manzana

In [ ]:
# export
def imputa_faltantes_manzana(censo, cols, metodo='ceros'):
    """ Regresa un df con los datos faltantes imputados a nivel manzana.
    
        params:
        
        cols: list: lista con las columnas en donde se deben imputar faltantes.
        metodo: método a usar para la imputación.
        NOTA: Por lo pronto sólo implementa dos métodos muy simples: llena con ceros 
        o aleatorio entre 0 y 3. En el futuro podríamos implementar mejores formas
    """
    if metodo == 'ceros':
        censo = censo.fillna(0)
    elif metodo == 'random':
        rand = pd.DataFrame(np.random.randint(0, 4, size=(censo.shape[0], len(cols))),
                            columns=cols, 
                            index=censo.index)
        censo.update(rand) 
    else:
        raise ValueError("imputacion debe ser ceros o random")
    return censo

In [ ]:
# Prueba funcionamiento normal
vars_pob = [v for v in diccionario['Nombre del Campo'].unique() 
            if (v.startswith('P') and v != 'PROM_HNV') ]
vars_viv = [v for v in diccionario['Nombre del Campo'].unique() if v.startswith('V')]
vars_viv.append('OCUPVIVPAR')
cols = vars_pob + vars_viv
imputados_ceros = imputa_faltantes_manzana(censo.head(), cols)
imputados_rand = imputa_faltantes_manzana(censo.head(), cols, metodo='random')
assert np.count_nonzero(imputados_rand.loc[:,cols].head().isna()) == 0
assert np.count_nonzero(imputados_ceros.loc[:,cols].head().isna()) == 0
# Prueba que arroje la excepción
# TODO

## agrega_en_unidades

In [ ]:
# export 
def agrega_en_unidades(censo, diciconario,
                       agregacion        = 'colonias', 
                       imputacion        = 'ceros',
                       umbral_faltantes  = 0.5):
    """ Agrega las variables del censo en las unidades espaciales especificadas.
    
        params:
        agregacion: str: colonias/cuadrantes o nombre del campo
        imputacion: str: ceros/random. método para rellenar los datos faltantes. 
                             ceros llena con ceros, random con un aleatorio entre 0 y 3 
                             (faltantes por secreto)
        umbral_faltantes float: Porcentaje de datos faltantes en una manzana para 
                                considerarla en el análisis
                            
        NOTA: Las columnas PROM_HNV, GRAPROES(F/M) se pierden porque 
        no hay forma de calcularlas.
    """
    vars_pob = [v for v in diccionario['Nombre del Campo'].unique() 
                if (v.startswith('P') and v != 'PROM_HNV') ]
    vars_viv = [v for v in diccionario['Nombre del Campo'].unique() if v.startswith('V')]
    vars_viv.append('OCUPVIVPAR')
    if agregacion == 'colonias':
        columna_agrega = 'colonia_cve'
    elif agregacion == 'cuadrantes':
        columna_agrega = 'cuadrante_id'
    else:
        columna_agrega = agregacion
    assert columna_agrega in censo.columns
    censo.dropna(thresh=umbral_faltantes*(len(vars_pob) + len(vars_viv)), inplace=True)
    censo = imputa_faltantes_manzana(censo, vars_pob + vars_viv, metodo=imputacion)
    censo = censo[[columna_agrega] + vars_pob + vars_viv].groupby(columna_agrega).sum()
    # Calculamos las columnas que requieren trato espacial
    censo['V_PROM_OCUP'] = censo['OCUPVIVPAR'].div(censo['VIVPAR_HAB'])
    censo['V_PROM_OCUP_C'] = censo['OCUPVIVPAR'].div(censo['VPH_1CUART'] + 2*censo['VPH_2CUART'] + 3*censo['VPH_3YMASC'])
    return censo

In [ ]:
agregado = agrega_en_unidades(censo, diccionario, imputacion='random')
agregado = agrega_en_unidades(censo, diccionario)
agregado

,POBTOT,POBFEM,POBMAS,P_0A2,P_0A2_F,P_0A2_M,P_3YMAS,P_3YMAS_F,P_3YMAS_M,P_5YMAS,...,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,OCUPVIVPAR,V_PROM_OCUP,V_PROM_OCUP_C
colonia_cve,,,,,,,,,,,,,,,,,,,,,
0,106.0,114.0,119.0,101.0,125.0,134.0,119.0,125.0,121.0,130.0,...,134.0,131.0,121.0,121.0,126.0,120.0,106.0,116.0,0.846715,0.157609
1,85.0,79.0,71.0,90.0,82.0,95.0,82.0,78.0,75.0,79.0,...,78.0,74.0,68.0,85.0,81.0,86.0,84.0,78.0,0.906977,0.150870
2,98.0,102.0,108.0,98.0,97.0,96.0,98.0,102.0,107.0,109.0,...,95.0,99.0,99.0,108.0,103.0,103.0,119.0,124.0,1.180952,0.193448
3,28.0,26.0,22.0,18.0,23.0,12.0,22.0,29.0,22.0,25.0,...,7.0,23.0,18.0,21.0,21.0,19.0,29.0,19.0,1.000000,0.136691
4,45.0,42.0,40.0,49.0,59.0,48.0,41.0,40.0,50.0,45.0,...,48.0,40.0,46.0,46.0,43.0,45.0,48.0,33.0,1.000000,0.132530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1820,109.0,98.0,105.0,95.0,96.0,106.0,71.0,92.0,94.0,90.0,...,84.0,101.0,91.0,106.0,93.0,94.0,92.0,80.0,0.879121,0.147874
1821,193.0,216.0,199.0,204.0,229.0,212.0,225.0,207.0,204.0,234.0,...,207.0,190.0,234.0,205.0,220.0,222.0,188.0,206.0,0.995169,0.161189
1822,4.0,4.0,8.0,9.0,6.0,8.0,6.0,8.0,8.0,8.0,...,5.0,7.0,6.0,2.0,7.0,8.0,5.0,5.0,0.555556,0.156250


## censo_a_tasas

In [ ]:
# export
def censo_a_tasas(censo, diccionario, agregacion='colonias', umbral_faltantes=0.5):
    """Convierte las variables del censo a tasas en la agregación seleccionada.

    Para las variables de población divide por población total.
    Para Hogares divide por TOTHOG.
    Para viviendas divide por el total de viviendas particulares 
     habitadas (VIVPAR_HAB)
    
    params:
    umbral_faltantes float: Porcentaje de datos faltantes en una manzana para 
    considerarla en el análisis
    """
    pob_col = 'POBTOT'
    hog_col = 'TOTHOG'
    viv_col = 'VIVPAR_HAB'
    vars_pob = [v for v in diccionario['Nombre del Campo'].unique() if v.startswith('P')]
    eliminar = ['POBTOT', 'PROM_HNV']
    vars_pob = [v for v in vars_pob if (v not in eliminar)]
    censo.dropna(thresh=umbral_faltantes*len(vars_pob), inplace=True)
    if agregacion == None:
        censo[vars_pob] = censo[vars_pob].div(censo[pob_col], axis=0)
    return censo